In [ ]:
# rdc_prep1 、履修履歴データの下準備、第一段階

In [ ]:
# 履修履歴データを準備する

#1. 履修履歴データの入手
#履修履歴データの入手（利用）を希望する場合は、株式会社大学成績センターに利用申請を行う。
#提供条件等の詳細は右のサイトで確認できる http://dscenter.co.jp/data/
#履修履歴データは就活年度別にまとめられている
#2019就活生データとは、2019年3月卒業、4月新入社する学生の残したデータを指す

#2. 履修履歴データの構成
#履修履歴データは、履修履歴マスデータ（全体データ）と該当講義名リストから構成される
#履修履歴マスデータ（全体データ）とは、学生ID番号、履修科目名、成績等の csv データで、１履修科目１行でまとめられている。
#履修履歴マスデータは、15万人分700万行など比較的ファイルサイズの大きい csv で提供される。
#該当講義名リストとは、RDCが定める「標準科目名」（2020年度現在120科目）と大学が自機関内で定めるところの開設科目名との対照リストである。
#該当講義名リストは、MS Excel ファイルで提供される。


#3.リネーム
#履修履歴データは日本語ファイル名で支給される。
#以下の処理においては次のようにリネームした
#2019就活生データ：「履修履歴マスデータ（全体データ）2019.csv」→ RDCdata2019_sjis_org.csv、※文字コードが sjis
#2020就活生データ：「履修履歴マスデータ（全体データ）2020.csv」→ RDCdata2020_utf8_org.csv、※文字コード utf8
#講義名リスト2019年版v1.2.xlsx → RDCsubjectlist_2019_12_org.xlsx'
#講義名リスト2020年版v1.1.xlsx → RDCsubjectlist_2020_11_org.xlsx'


#4.ファイルの配置
#次のように元データのファイルを整理格納して以下の処理を行う
#
#作業フォルダ（ルート）
#┗ RDCフォルダ：計算に使うデータを格納
#  ┗ orgフォルダ：支給された履修履歴データを格納（上記 #3 のリネーム済み）


#5.文字コードの変換（2019就活生データのみ必要）
import codecs
#sf = codecs.open('RDC/RDCdata2019_sjis_org.csv', 'r', encoding='shift_jis')
#shift_jis で読み込むと0xfa エラーが出るので cp932 で読み込む
sf = codecs.open('RDC/org/RDCdata2019_sjis_org.csv', 'r', encoding='cp932')
uf = codecs.open('RDC/org/RDCdata2019_utf8_org.csv', 'w', encoding='utf_8')
for line in sf:
    uf.write(line)
sf.close()
uf.close()

In [ ]:
# 履修履歴データを見てみる

#履修履歴データ全体を読み込み
import pandas as pd
rdcdata_org = pd.read_csv('RDC/org/RDCdata2020_utf8_org.csv', encoding='utf_8', dtype={"univ_cd":str, "dep_cd":str})

# --
#結果の確認
# --
#冒頭と末尾を表示（各5行）
print(rdcdata_org)
#列名を表示
print(rdcdata_org.columns)
#行数をカウント
rdcdata_org.count()
# --
#メモ
# --
#RDCdata2019_utf8.csv の loginnum は、8,277,757
#RDCdata2020_utf8.csv の loginnum は、8,588,983

In [ ]:
# 履修履歴データの列名を変更する
#10列目の replace は、履修科目名が入るので、local_course_name とする
rdcdata = rdcdata_org.rename(columns={'replace':'local_course_name'})
rdcdata.to_csv('RDC/RDCdata2020_utf8.csv', header=True, index=False)

# --
#結果の確認
# --
#冒頭と末尾を表示（各5行）
print(rdcdata)
#列名を表示
print(rdcdata.columns)
#行数をカウント
print(rdcdata.count())

In [ ]:
# 該当講義名リストの列名を変更する
# 科目名リストと履修データの列名を揃える

import pandas as pd
excel_file = 'RDC/org/RDCsubjectlist_2020_11_org.xlsx'
sheet_name = '該当講義名リスト2020'
df = pd.read_excel(excel_file,sheet_name)
df_new = df.rename(columns={
    '系統':'course_type_name',
    'ＮＯ':'global_course_cd',
    '基準科目名':'global_course_name',
    '大学名':'univ_name',
    '学部名':'dep_name',
    '学科名':'sub_name',
    '講義名':'local_course_name'
})
excel_file = 'RDC/RDCsubjectlist_2020_11.xlsx'
sheet_name = 'RDCsubjectlist2020'
df_new.to_excel(excel_file, sheet_name, index=False, header=True)